In [1]:
%pip install -r requirements.txt 

  Using cached pyspark-3.5.1.tar.gz (317.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pandas-2.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached polars-0.20.19-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached SQLAlchemy-2.0.29-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.0 MB/s eta 0:

In [1]:
import numpy as np #scientific computing
import pandas as pd #data manipulation and analysis
import os #to read env variables
import time
from pyspark.sql import SparkSession # to interact with db
from helpers import *
from spark_helpers import *
from fastparquet import write


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 09:59:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/18 09:59:11 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/18 09:59:11 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/18 09:59:12 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/18 09:59:12 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jay@10.160.0.4
24/04/18 09:59:13 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


## Preprocessing
#### Mapping 65 patients to 2000 

In [2]:
csv_file_path = '../processedFinal2'
num_duplicates = 30
all_dataframes = process_data(csv_file_path, num_duplicates)
print(len(all_dataframes))


100%|██████████| 30/30 [00:00<00:00, 3025.76it/s]

2015


In [5]:
csv_file_path = '../semi_processed2'
num_duplicates = 30
all_semis = process_data(csv_file_path, num_duplicates)
print(len(all_semis))

/home/jaikar/genepower/genepowerx/helpers.py:31: DtypeWarning: Columns (74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/home/jaikar/genepower/genepowerx/helpers.py:31: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


### Writing to Parquet

In [3]:
# Concatenate all data frames in all_dataframes
concatenated_df = pd.concat(all_dataframes)

# Write the concatenated data frame to a Parquet file
concatenated_df.to_parquet('spark-warehouse/genepowerx.db/db.parquet')

In [ ]:
semi_csv_path = "../semi_processed"
semi_dest = 'spark-warehouse/genepowerx.db/semi_processed.parquet'
process_semi(semi_csv_path,2,semi_dest)

## Creating and adding to Database

In [4]:

# Set the current database to "genepowerx"
sparkQ("SHOW DATABASES")

start = time.time()

sparkQ("CREATE TABLE IF NOT EXISTS model1 USING parquet OPTIONS  (path 'db.parquet') ")
# sparkQ("CREATE TABLE IF NOT EXISTS model3 USING parquet OPTIONS  (path 'semi_processed.parquet') ")

end = time.time()
print("Insertion time:",end - start)

24/04/11 15:08:50 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/04/11 15:08:50 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/04/11 15:08:50 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/11 15:08:50 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/11 15:08:50 WARN HiveExternalCatalog: Could not persist `spark_catalog`.`genepowerx`.`model1` in a Hive compatible way. Persisting it into Hive metastore in Spark SQL specific format.
org.apache.hadoop.hive.ql.metadata.HiveException: MetaException(message:file:/home/jay/genepowerx/spark-warehouse/genepowerx.db/db.parquet is not a directory or unable to create one)
	at org.apache.hadoop.hive.ql.metadata.Hive.createTable(Hive.java:869)
	at org.apache.hadoop.hive.ql.metadata.Hive.createTable(Hive.java:874)
	at org.apache.spark.sql.hive.cl

Insertion time: 1.1145899295806885


In [5]:
queries = [
    "SELECT COUNT (DISTINCT patient_id) FROM model1 ",
    "SELECT COUNT(*) FROM model1 WHERE zygosity = 'Homozygous'",
    "SELECT COUNT(*) FROM model1 WHERE gene LIKE 'PADI4' AND zygosity = 'Heterozygous'",
    "SELECT COUNT(*) FROM model1 WHERE gene LIKE 'PADI4'"
]
for query in queries:
    print(query)
    result, timeTaken = runQ(query,sparkQ)
    print("Time for this query was: ", timeTaken*1000, "ms\n")

SELECT COUNT (DISTINCT patient_id) FROM model1 
Time for this query was:  1460.6211185455322 ms

SELECT COUNT(*) FROM model1 WHERE zygosity = 'Homozygous'
Time for this query was:  405.3916931152344 ms

SELECT COUNT(*) FROM model1 WHERE gene LIKE 'PADI4' AND zygosity = 'Heterozygous'
Time for this query was:  306.06555938720703 ms

SELECT COUNT(*) FROM model1 WHERE gene LIKE 'PADI4'
Time for this query was:  223.17075729370117 ms



In [2]:
query = "SELECT COUNT (DISTINCT patient_id) FROM model3"
print(query)
result, timeTaken = runQ(query,sparkQ)
print("Time for this query was: ", timeTaken*1000, "ms\n")

SELECT COUNT (DISTINCT patient_id) FROM model3


+--------------------------+
|count(DISTINCT patient_id)|
+--------------------------+
|                        15|
+--------------------------+

Time for this query was:  3260.6968879699707 ms



In [6]:
stop()